In [1]:
from azureml.core import Workspace, Experiment

name = "<Workspace Name>"
subid = "<Subscription ID>"
rg = "<Resource Group>"

ws = Workspace.get(name=name,subscription_id=subid,resource_group=rg)
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_cluster = ComputeTarget(ws,"ml-clstr")
except:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",min_nodes=0,max_nodes=4,idle_seconds_before_scaledown=2400)
    compute_cluster = ComputeTarget.create(ws,"ml-clstr",compute_cluster_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C':uniform(0.05,30),
    '--max_iter':choice(40,60,80,100,120)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=4,slack_factor=0.2,)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=".",
    compute_target=compute_cluster,
    entry_script="train_new.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
primary_metric_name="Accuracy",
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=30,
max_concurrent_runs=4,
policy=policy,
estimator=est)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.widgets import RunDetails

hd_submit = hyperdrive_details = exp.submit(config=hyperdrive_config)

RunDetails(hd_submit).show()

hd_submit.wait_for_completion(show_output=True)

In [5]:
# import joblib
# Get your best run and save the model from that run.
best_run_hd = hd_submit.get_best_run_by_primary_metric()
best_model_hd = best_run_hd.register_model(model_name = 'udacityprojecthd',model_path='.')

In [6]:
best_model_hd

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
tab_data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=tab_data_url)

In [8]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

df = pd.concat([x,y],axis=1)
df.head()

In [9]:
def_datastore = ws.get_default_datastore()
print(def_datastore)

In [10]:
from azureml.data.datapath import DataPath
data_final = TabularDatasetFactory.register_pandas_dataframe(dataframe = df,
                                                       target=DataPath(datastore=def_datastore,path_on_datastore='./automl/data'),
                                                       name='bankmarketing')

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_final,
    label_column_name='y',
    n_cross_validations=8,
    compute_target=compute_cluster,
    max_cores_per_iteration=-1,
    max_concurrent_iterations = 4,
    enable_early_stopping = True
    )

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config,show_output=True)

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run = automl_run.get_best_child()

automl_model = automl_best_run.register_model(model_name = 'automl_best_model',model_path='.')


In [14]:
automl_model

In [15]:
# Delete Cluster
compute_cluster.delete()